## 웹 크롤링을 통한 데이터 수집
네이버 쇼핑에서 원하는 검색어를 입력하여 나온 상품들의 5페이지까지의 상품들에 대한 정보를 가져온다.

In [6]:
import requests
import bs4
import pandas as pd
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
import time
from tqdm.notebook import tqdm
import warnings
warnings.simplefilter('ignore')

In [42]:
query = "애플"

titles = []
prices = []
review_counts = []
buy_counts = []
published_dates = []
favorites = []

driver = Chrome('./chromedriver')

for page_no in tqdm(range(1, 6)):  # 5페이지
    page_url = f"https://search.shopping.naver.com/search/all?frm=NVSCTAB&origQuery={query}&pagingIndex={page_no}&pagingSize=40&productSet=total&query={query}&sort=rel&timestamp=&viewType=list"
    driver.get(page_url)
    time.sleep(1)
    
    for scroll_down in range(10): # 스크롤 10번정도 하면 끝까지 내려감
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)') # 자바스크립트 쿼리
        time.sleep(0.5)
    
    list_basis = driver.find_element(By.CLASS_NAME, 'list_basis')
    item_list = list_basis.find_elements(By.CLASS_NAME, 'basicList_inner__xCM3J')
    
    items = []
    for i in tqdm(range(len(item_list))):
        item = item_list[i]
        title = item.find_element(By.CLASS_NAME, 'basicList_title__VfX3c').text
        titles.append(title)
        
        price = item.find_element(By.CLASS_NAME, 'price_num__S2p_v').text.replace(',', '')[:-1]
        prices.append(int(price))
        
        footer = item.find_element(By.CLASS_NAME, 'basicList_etc_box__5lkgg')
        footer_text = footer.text
        
        try:
            reviews = footer.find_elements(By.CLASS_NAME, 'basicList_num__sfz3h')
            if "리뷰" in footer_text:
                review_counts.append(int(reviews[0].text.replace(',', '')))
            else:
                review_counts.append(0)
            if "구매건수" in footer_text:
                buy_counts.append(int(reviews[1].text.replace(',', '')))
            else:
                buy_counts.append(0)
            if "등록일" in footer_text:
                published_dates.append(footer.find_element(By.CSS_SELECTOR, 'span.basicList_etc__LSkN_').text[4:])
            else:
                published_dates.append('')
            if "찜하기" in footer_text:
                favorites.append(int(reviews[-1].text.replace(',', '')))
            else:
                favorites.append(0)
        except IndexError:
            continue

driver.close()
        

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

In [34]:
print(len(titles), len(prices), len(review_counts), len(published_dates), len(favorites))

200 200 200 200 200


In [47]:
result = pd.DataFrame({"title": titles,
                      "price": prices,
                      "review_counts": review_counts,
                      "published_date": published_dates,
                      "favorites": favorites})
result

,title,price,review_counts,published_date,favorites
0,Apple 에어팟 프로 2세대 (MQD83KH/A),303000,7836,2022.09.,839
1,Apple 에어팟 3세대 맥세이프 충전 케이스 모델 (MME73KH/A),241710,12178,2021.10.,1512
2,Apple 에어팟 프로 1세대 2021년형 맥세이프 호환 (MLWK3KH/A),254940,10341,2021.10.,1609
3,애플 에어팟 프로 2세대 에어팟프로2 국내정품 MQD83KH/A 당일배송,331900,527,2022.10.,438
4,해외Apple 에어팟 프로 2세대 (MQD83KH/A),205000,688,2022.09.,387
...,...,...,...,...,...
195,애플 에어팟 2세대 with 충전 케이스 A2032 A2031 A1602 + Ali...,267400,0,2022.12.,0
196,해외애플 AirPods 2세대 왼쪽-A2031(L) 교체단품 호환 새품,58340,0,2022.11.,1
197,해외애플 파워비츠 프로 Powerbeats Pro 무선이어폰 익스트림블루투스이어폰,276000,0,2022.10.,0
198,애플 Airpods 2 에어팟2세대 MV7N2KH/A 유선충전,231100,0,2022.12.,0


In [48]:
result.to_csv("review_data.csv", index=False)